# scVI

- **Creator**: Sebastian Birk (<sebastian.birk@helmholtz-munich.de>).
- **Affiliation:** Helmholtz Munich, Institute of Computational Biology (ICB), Talavera-López Lab
- **Date of Creation:** 05.01.2023
- **Date of Last Modification:** 09.03.2023

- The scVI source code is available at https://github.com/scverse/scvi-tools.
- The corresponding publication is "Lopez, R., Regier, J., Cole, M. B., Jordan, M. I. & Yosef, N. Deep generative modeling for single-cell transcriptomics. Nat. Methods 15, 1053–1058 (2018)".
- The workflow of this notebook follows the tutorial from https://docs.scvi-tools.org/en/stable/tutorials/notebooks/api_overview.html.
- The authors use raw counts as input to scVI. Therefore, we also use raw counts (stored in adata.X).

## 1. Setup

### 1.1 Import Libraries

In [1]:
import os
import time
from datetime import datetime

import scvi
import scanpy as sc
import scipy.sparse as sp
import squidpy as sq
import matplotlib.pyplot as plt
import numpy as np

[rank: 0] Global seed set to 0


### 1.2 Define Parameters

In [2]:
model_name = "scvi"
latent_key = f"{model_name}_latent"
leiden_resolution = 0.5 # used for Leiden clustering of latent space
random_seed = 0 # used for Leiden clustering

### 1.3 Run Notebook Setup

In [3]:
sc.set_figure_params(figsize=(6, 6))

In [4]:
# Get time of notebook execution for timestamping saved artifacts
now = datetime.now()
current_timestamp = now.strftime("%d%m%Y_%H%M%S")

### 1.4 Configure Paths and Directories

In [5]:
data_folder_path = "../../datasets/srt_data/gold/"
figure_folder_path = f"../../figures"

## 2. scVI Model

### 2.1 Define Training Function

In [6]:
def train_scvi_models(dataset,
                      cell_type_key,
                      adata_new=None,
                      n_start_run=1,
                      n_end_run=10,
                      n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20],
                      plot_latent_umaps=False):
    # Configure figure folder path
    dataset_figure_folder_path = f"{figure_folder_path}/{dataset}/method_benchmarking/" \
                                 f"{model_name}/{current_timestamp}"
    os.makedirs(dataset_figure_folder_path, exist_ok=True)
    
    # Create new adata to store results from training runs in storage-efficient way
    if adata_new is None:
        adata_original = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")
        adata_new = sc.AnnData(sp.csr_matrix(
            (adata_original.shape[0], adata_original.shape[1]),
            dtype=np.float32))
        adata_new.var_names = adata_original.var_names
        adata_new.obs_names = adata_original.obs_names
        adata_new.obs["cell_type"] = adata_original.obs[cell_type_key].values
        adata_new.obsm["spatial"] = adata_original.obsm["spatial"]
        del(adata_original)
    
    model_seeds = list(range(10))
    for run_number, n_neighbors in zip(np.arange(n_start_run, n_end_run+1), n_neighbor_list):
        # n_neighbors is here only used for the latent neighbor graph construction used for
        # UMAP generation and clustering as scVI is not a spatial method
        # Load data
        adata = sc.read_h5ad(data_folder_path + f"{dataset}.h5ad")
        
        start_time = time.time()
        
        scvi.settings.seed = model_seeds[run_number-1]

        # Setup adata
        scvi.model.SCVI.setup_anndata(adata,
                                      layer="counts")

        # Initialize model
        model = scvi.model.SCVI(adata)

        # Train model
        model.train()

        adata.obsm[latent_key] = model.get_latent_representation()
        
        # Measure time for model training
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Duration of model training in run {run_number}: "
              f"{int(hours)} hours, {int(minutes)} minutes and {int(seconds)} seconds.")
        adata_new.uns[f"{model_name}_model_training_duration_run{run_number}"] = (
            elapsed_time)
        
        if plot_latent_umaps:
            # Use scVI latent space for UMAP generation
            sc.pp.neighbors(adata,
                            use_rep=latent_key,
                            n_neighbors=n_neighbors)
            sc.tl.umap(adata)
            fig = sc.pl.umap(adata,
                             color=[cell_type_key],
                             title="Latent Space with Cell Types: scVI",
                             return_fig=True)
            fig.savefig(f"{dataset_figure_folder_path}/latent_{model_name}"
                        f"_cell_types_run{run_number}.png",
                        bbox_inches="tight")

            # Compute latent Leiden clustering
            sc.tl.leiden(adata=adata,
                         resolution=leiden_resolution,
                         random_state=random_seed,
                         key_added=f"latent_{model_name}_leiden_{str(leiden_resolution)}")

            # Create subplot of latent Leiden cluster annotations in physical and latent space
            fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(6, 12))
            title = fig.suptitle(t="Latent and Physical Space with Leiden Clusters: scVI")
            sc.pl.umap(adata=adata,
                       color=[f"latent_{model_name}_leiden_{str(leiden_resolution)}"],
                       title=f"Latent Space with Leiden Clusters",
                       ax=axs[0],
                       show=False)
            sq.pl.spatial_scatter(adata=adata,
                                  color=[f"latent_{model_name}_leiden_{str(leiden_resolution)}"],
                                  title=f"Physical Space with Leiden Clusters",
                                  shape=None,
                                  ax=axs[1])

            # Create and position shared legend
            handles, labels = axs[0].get_legend_handles_labels()
            lgd = fig.legend(handles, labels, bbox_to_anchor=(1.25, 0.9185))
            axs[0].get_legend().remove()
            axs[1].get_legend().remove()

            # Adjust, save and display plot
            plt.subplots_adjust(wspace=0, hspace=0.2)
            fig.savefig(f"{dataset_figure_folder_path}/latent_physical_comparison_"
                        f"{model_name}_run{run_number}.png",
                        bbox_extra_artists=(lgd, title),
                        bbox_inches="tight")
            plt.show()
        
        # Store latent representation
        adata_new.obsm[latent_key + f"_run{run_number}"] = adata.obsm[latent_key]

        # Store intermediate adata to disk
        adata_new.write(f"{data_folder_path}/{dataset}_{model_name}.h5ad")  

    # Store final adata to disk
    adata_new.write(f"{data_folder_path}/{dataset}_{model_name}.h5ad") 

### 2.2 Train Models on Benchmarking Datasets

In [ ]:
train_scvi_models(dataset="seqfish_mouse_organogenesis_embryo2",
                  cell_type_key="celltype_mapped_refined",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [ ]:
train_scvi_models(dataset="vizgen_merfish_mouse_liver",
                  cell_type_key="Cell_Type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [ ]:
# ~10% sample of the original dataset
train_scvi_models(dataset="vizgen_merfish_mouse_liver_sample",
                  cell_type_key="Cell_Type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [ ]:
train_scvi_models(dataset="starmap_plus_mouse_cns",
                  cell_type_key="Main_molecular_cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [ ]:
# ~20% sample of the original dataset
train_scvi_models(dataset="starmap_plus_mouse_cns_sample",
                  cell_type_key="Main_molecular_cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [ ]:
train_scvi_models(dataset="nanostring_cosmx_human_nsclc",
                  cell_type_key="cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [ ]:
# ~20% sample of the original dataset
train_scvi_models(dataset="nanostring_cosmx_human_nsclc_sample",
                  cell_type_key="cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

In [7]:
train_scvi_models(dataset="slideseqv2_mouse_hippocampus",
                  cell_type_key="cell_type",
                  adata_new=None,
                  n_start_run=1,
                  n_end_run=10,
                  n_neighbor_list=[4, 4, 8, 8, 12, 12, 16, 16, 20, 20])

[rank: 0] Global seed set to 0
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/data/fields/_layer_field.py:91: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/191:   0%|          | 0/191 [00:00<?, ?it/s]

/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:481: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(
/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:481: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(


Epoch 2/191:   1%|          | 1/191 [00:12<38:37, 12.20s/it, loss=210, v_num=1]

/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:481: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(
/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:481: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(


...

`Trainer.fit` stopped: `max_epochs=191` reached.


Epoch 191/191: 100%|██████████| 191/191 [06:34<00:00,  2.07s/it, loss=189, v_num=1]
Duration of model training in run 6: 0 hours, 6 minutes and 35 seconds.


[rank: 0] Global seed set to 6
/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/data/fields/_layer_field.py:91: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/191:   0%|          | 0/191 [00:00<?, ?it/s]

/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:481: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(
/home/sebastian.birk/.local/lib/python3.9/site-packages/scvi/distributions/_negative_binomial.py:481: UserWarning: The value argument must be within the support of the distribution
  warnings.warn(
/home/sebastian.birk/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

KeyboardInterrupt

